# Decoding Social Intent from Neural Oscillations
## Exploratory Data Analysis

**COGS 118C — Signal Processing Course Project**

**Research Question:** Can we classify whether an animal is interacting socially vs. exploring alone based on the spectral features of its neural calcium signals?

---

### Goals of this notebook

1. **Download & load** the three core datasets: calcium traces, behavioral labels, session metadata
2. **Inspect** data shapes, sampling rates, and quality across all 18 sessions
3. **Visualize** raw calcium signals with behavioral epoch overlays
4. **Preliminary spectral analysis** — PSD, spectrograms, wavelets
5. **Compare** spectral features between social vs. solo epochs
6. **Sanity-check classification** with permutation test

### Data sources

| File | Format | Content |
|------|--------|---------|
| `calcium.00.h5` | HDF5 | 18 sessions of calcium traces (`C` key, frames x neurons), **30 fps** |
| `social_bouts.00.h5` | HDF5 | 18 sessions of boolean behavioral labels, **25 fps** |
| `SI3_2022_Entrance_Frames.xlsx` | Excel | Session metadata (animal, isolation condition, entry frame) |

---
## 1. Setup & Dependencies

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import h5py
from pathlib import Path

from scipy.signal import welch, butter, filtfilt, stft
from scipy.stats import mannwhitneyu
import pywt

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score, roc_auc_score

from tqdm import tqdm

sns.set_theme(style='whitegrid', context='notebook', font_scale=1.1)
plt.rcParams['figure.figsize'] = (14, 5)
plt.rcParams['figure.dpi'] = 100
np.random.seed(42)
print('All imports successful.')

---
## 2. Data Download & Loading

Three files from Google Drive, downloaded with `gdown`. The Drive file IDs were extracted from the EDGE source notebooks.

In [ ]:
import gdown

DATA_DIR = Path('../data/raw')
DATA_DIR.mkdir(parents=True, exist_ok=True)

FILES = {
    'calcium.00.h5':                 '1UthpsskvkHbKKDsbQjUxyVN4Xkd-ZJuN',   # ~250 MB
    'social_bouts.00.h5':            '1Mh8oGKNyKpT5WS0Wu92SULFFanvqmSMf',   # ~510 KB
    'SI3_2022_Entrance_Frames.xlsx': '1POpRqpA_QaWfZhxswQvLSs9uBnnHrmhZ',   # ~10 KB
}

for filename, file_id in FILES.items():
    out_path = DATA_DIR / filename
    if out_path.exists():
        size_mb = out_path.stat().st_size / 1024 / 1024
        print(f'  [skip] {filename} ({size_mb:.1f} MB)')
    else:
        print(f'  Downloading {filename}...')
        gdown.download(f'https://drive.google.com/uc?id={file_id}', str(out_path), quiet=False)
        size_mb = out_path.stat().st_size / 1024 / 1024
        print(f'  [ok] {filename} ({size_mb:.1f} MB)')

### 2.1 Load session metadata

In [ ]:
entrances = pd.read_excel(DATA_DIR / 'SI3_2022_Entrance_Frames.xlsx')
n_sessions = len(entrances)

print(f'Sessions: {n_sessions}')
print(f'Columns: {list(entrances.columns)}')
print(f'\nIsolation conditions:')
print(entrances['Isolation Length'].value_counts().to_string())
print()
entrances

### 2.2 Load behavioral labels (25 fps)

Boolean arrays from `social_bouts.00.h5`. Each session group has:
- `is_social` = any social interaction (sender OR receiver)
- `is_social_sender` / `is_social_receiver` = directional
- `is_ag_sniffed`, `is_ag_sniffing`, `is_of_sniffed`, `is_of_sniffing`, `is_touched`, `is_touching`

In [ ]:
BEHAVIOR_FPS = 25

behavior_keys = [
    'is_ag_sniffed', 'is_ag_sniffing', 'is_of_sniffed', 'is_of_sniffing',
    'is_social', 'is_social_receiver', 'is_social_sender',
    'is_touched', 'is_touching',
]

behavior = []
with h5py.File(DATA_DIR / 'social_bouts.00.h5', 'r') as f:
    print(f'Top-level keys: {list(f.keys())}\n')
    for sess in range(n_sessions):
        sd = {}
        for key in behavior_keys:
            sd[key] = f[f'session_{sess}'][key][:]
        behavior.append(sd)
        nf = len(sd['is_social'])
        pct = sd['is_social'].mean() * 100
        print(f'  Session {sess:2d}: {nf:6d} frames, '
              f'{pct:5.1f}% social, '
              f'isolation={entrances.iloc[sess]["Isolation Length"]}')

print(f'\nLoaded {len(behavior)} sessions.')

### 2.3 Load calcium imaging data (30 fps)

Each session has a `C` matrix of shape `(n_frames, n_neurons)` in `calcium.00.h5`.

In [ ]:
IMAGING_FPS = 30

imaging = []
with h5py.File(DATA_DIR / 'calcium.00.h5', 'r') as f:
    print(f'Top-level keys: {list(f.keys())}\n')
    for sess in range(n_sessions):
        C = f[f'session_{sess}']['C'][:]
        imaging.append(C)
        print(f'  Session {sess:2d}: C shape = {str(C.shape):>16s}  '
              f'({C.shape[1]:3d} neurons, {C.shape[0]/IMAGING_FPS:.1f}s)')

print(f'\nLoaded {len(imaging)} sessions.')
print(f'Imaging FPS: {IMAGING_FPS} | Nyquist: {IMAGING_FPS/2} Hz')

### 2.4 Align behavior to imaging & crop to post-entry

Behavior is at **25 fps**, imaging at **30 fps**. We:
1. Convert the intruder entry frame from behavior fps to imaging fps
2. Resample behavior labels (nearest-neighbor for boolean data)
3. Crop both signals to start at the entry frame
4. Trim to common length

In [ ]:
def align_session(calcium_C, beh_dict, entry_beh, beh_fps, img_fps):
    """Align behavior labels to calcium imaging for one session."""
    entry_img = int(entry_beh * (img_fps / beh_fps))
    if entry_img >= calcium_C.shape[0]:
        return None, None

    cal = calcium_C[entry_img:]
    beh = beh_dict['is_social'][int(entry_beh):]

    # Nearest-neighbor resample: beh_fps -> img_fps
    n_beh = len(beh)
    target = int(n_beh * (img_fps / beh_fps))
    idx = np.round(np.linspace(0, n_beh - 1, target)).astype(int)
    beh_rs = beh[idx]

    common = min(len(cal), len(beh_rs))
    return cal[:common], beh_rs[:common]


aligned_calcium = []
aligned_behavior = []
session_info = []

for i in range(n_sessions):
    entry = int(entrances.iloc[i]['Int_Entry'])
    cal, beh = align_session(imaging[i], behavior[i], entry, BEHAVIOR_FPS, IMAGING_FPS)
    if cal is None:
        print(f'  Session {i:2d}: SKIPPED')
        continue

    aligned_calcium.append(cal)
    aligned_behavior.append(beh)
    info = {
        'session_idx': i,
        'animal': entrances.iloc[i]['Animal'],
        'isolation': entrances.iloc[i]['Isolation Length'],
        'n_frames': len(cal),
        'n_neurons': cal.shape[1],
        'duration_s': len(cal) / IMAGING_FPS,
        'social_frac': beh.mean(),
    }
    session_info.append(info)
    print(f'  Session {i:2d}: {info["n_frames"]:6d} frames, '
          f'{info["n_neurons"]:3d} neurons, '
          f'{info["duration_s"]:.0f}s, '
          f'{info["social_frac"]*100:.1f}% social, '
          f'{info["isolation"]}')

session_df = pd.DataFrame(session_info)
print(f'\nAligned {len(aligned_calcium)} / {n_sessions} sessions.')

---
## 3. Data Quality & Basic Statistics

In [ ]:
print(session_df.to_string(index=False))
print(f'\nTotal neurons: {session_df["n_neurons"].sum()}')
print(f'Total time: {session_df["duration_s"].sum()/60:.1f} min')
print(f'Mean social: {session_df["social_frac"].mean()*100:.1f}%')

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

iso_colors = {'GH (7d)': '#3498db', 'GH (24hr)': '#2ecc71',
              '24hr': '#f39c12', '7d': '#e74c3c'}
colors = session_df['isolation'].map(iso_colors).fillna('#95a5a6')

axes[0].bar(range(len(session_df)), session_df['social_frac']*100, color=colors)
axes[0].set_xlabel('Session'); axes[0].set_ylabel('% Social')
axes[0].set_title('Social fraction per session')
axes[0].axhline(50, color='gray', ls='--', lw=0.8)

axes[1].bar(range(len(session_df)), session_df['n_neurons'], color=colors)
axes[1].set_xlabel('Session'); axes[1].set_ylabel('Neurons')
axes[1].set_title('Neuron count per session')

for cond in session_df['isolation'].unique():
    sub = session_df[session_df['isolation']==cond]
    axes[2].scatter([cond]*len(sub), sub['social_frac']*100,
                    s=80, alpha=0.7, color=iso_colors.get(cond, 'gray'))
axes[2].set_ylabel('% Social')
axes[2].set_title('Social fraction by isolation condition')
axes[2].tick_params(axis='x', rotation=20)

plt.tight_layout(); plt.show()

---
## 4. Calcium Signal Visualization

In [ ]:
# Pick session closest to 30% social
ex = session_df['social_frac'].sub(0.3).abs().idxmin()
ex_cal = aligned_calcium[ex]
ex_beh = aligned_behavior[ex]
ex_info = session_info[ex]
t = np.arange(len(ex_cal)) / IMAGING_FPS

print(f'Example: session {ex_info["session_idx"]} | '
      f'{ex_info["n_neurons"]} neurons | {ex_info["duration_s"]:.0f}s | '
      f'{ex_info["social_frac"]*100:.1f}% social | {ex_info["isolation"]}')

In [ ]:
def shade_social(ax, beh, t):
    """Red shading for social epochs."""
    mask = beh.astype(bool)
    d = np.diff(mask.astype(int))
    starts = np.where(d == 1)[0] + 1
    ends = np.where(d == -1)[0] + 1
    if mask[0]:  starts = np.concatenate([[0], starts])
    if mask[-1]: ends = np.concatenate([ends, [len(mask)]])
    for s, e in zip(starts, ends):
        ax.axvspan(t[s], t[min(e, len(t)-1)], alpha=0.15, color='#e74c3c')


n_show = 8
top_neurons = np.argsort(ex_cal.std(axis=0))[-n_show:][::-1]

fig, axes = plt.subplots(n_show+1, 1, figsize=(18, 2.2*(n_show+1)),
                         sharex=True,
                         gridspec_kw={'height_ratios': [1]*n_show + [0.5]})

for i, ni in enumerate(top_neurons):
    axes[i].plot(t, ex_cal[:, ni], lw=0.4, color='#2c3e50')
    shade_social(axes[i], ex_beh, t)
    axes[i].set_ylabel(f'N{ni}', fontsize=9)
    axes[i].tick_params(labelsize=8)

axes[-1].fill_between(t, ex_beh.astype(float), step='pre', alpha=0.7, color='#e74c3c')
axes[-1].set_yticks([0,1]); axes[-1].set_yticklabels(['Solo','Social'])
axes[-1].set_xlabel('Time (s)')
axes[0].set_title(f'Session {ex_info["session_idx"]} | Top {n_show} neurons (red=social)', fontsize=12)

plt.tight_layout(); plt.show()

In [ ]:
print(f'Calcium stats (session {ex_info["session_idx"]}):\n')
print(f'  Shape:   {ex_cal.shape} (frames x neurons)')
print(f'  Mean:    {ex_cal.mean():.4f}')
print(f'  Std:     {ex_cal.std():.4f}')
print(f'  Range:   [{ex_cal.min():.4f}, {ex_cal.max():.4f}]')
print(f'  Nyquist: {IMAGING_FPS/2} Hz')

stds = ex_cal.std(axis=0)
print(f'\nPer-neuron std: [{stds.min():.4f}, {stds.max():.4f}]')
print(f'Low-activity (std < median/5): {(stds < np.median(stds)/5).sum()}/{ex_cal.shape[1]}')

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 4))

axes[0].hist(ex_cal.ravel(), bins=100, color='#3498db', alpha=0.7, edgecolor='white')
axes[0].set_xlabel('Calcium (a.u.)'); axes[0].set_ylabel('Count')
axes[0].set_title('Distribution of calcium values')

axes[1].bar(range(len(stds)), np.sort(stds)[::-1], color='#2ecc71', width=1)
axes[1].set_xlabel('Neuron (sorted)'); axes[1].set_ylabel('Std')
axes[1].set_title('Activity level per neuron')

nc = min(30, ex_cal.shape[1])
corr = np.corrcoef(ex_cal[:, :nc].T)
im = axes[2].imshow(corr, cmap='RdBu_r', vmin=-1, vmax=1, aspect='auto')
axes[2].set_xlabel('Neuron'); axes[2].set_ylabel('Neuron')
axes[2].set_title(f'Pairwise correlation (first {nc})')
plt.colorbar(im, ax=axes[2], shrink=0.8)

plt.tight_layout(); plt.show()

### 4.1 Photobleaching check

In [ ]:
mean_trace = ex_cal.mean(axis=1)
fig, axes = plt.subplots(1, 2, figsize=(16, 4))

axes[0].plot(t, mean_trace, lw=0.8, color='#2c3e50')
slope, intercept = np.polyfit(t, mean_trace, 1)
axes[0].plot(t, slope*t + intercept, '--r', lw=1.5, label=f'slope={slope:.2e}/s')
axes[0].set_xlabel('Time (s)'); axes[0].set_ylabel('Mean calcium')
axes[0].set_title('Population mean - photobleaching check'); axes[0].legend()

win = int(30 * IMAGING_FPS)
rolling = pd.Series(mean_trace).rolling(window=win, center=True).mean()
axes[1].plot(t, rolling, lw=1.2, color='#e74c3c')
shade_social(axes[1], ex_beh, t)
axes[1].set_xlabel('Time (s)'); axes[1].set_ylabel('30s rolling mean')
axes[1].set_title('Slow baseline drift (red=social)')

plt.tight_layout(); plt.show()
if abs(slope) > 1e-5:
    print(f'Baseline drift detected (slope={slope:.2e}/s). Detrending recommended.')
else:
    print('No significant baseline drift.')

---
## 5. Behavioral Epoch Structure

In [ ]:
def get_epoch_durations(labels, fps):
    d = np.diff(labels.astype(int))
    bounds = np.where(d != 0)[0] + 1
    starts = np.concatenate([[0], bounds])
    ends = np.concatenate([bounds, [len(labels)]])
    labs = np.array([labels[s] for s in starts])
    durs = (ends - starts) / fps
    return starts, ends, labs, durs

_, _, ep_labs, ep_durs = get_epoch_durations(ex_beh, IMAGING_FPS)
soc_d = ep_durs[ep_labs == 1]
sol_d = ep_durs[ep_labs == 0]

print(f'Session {ex_info["session_idx"]}:')
print(f'  Epochs: {len(ep_durs)}')
print(f'  Social: {len(soc_d)}, mean={soc_d.mean():.2f}s, range=[{soc_d.min():.2f}, {soc_d.max():.2f}]')
print(f'  Solo:   {len(sol_d)}, mean={sol_d.mean():.2f}s, range=[{sol_d.min():.2f}, {sol_d.max():.2f}]')

fig, axes = plt.subplots(1, 3, figsize=(18, 4))

sf = ex_beh.mean()
axes[0].pie([sf, 1-sf], labels=['Social','Solo'],
            colors=['#e74c3c','#3498db'], autopct='%1.1f%%', startangle=90)
axes[0].set_title('Class balance')

mx = min(np.percentile(ep_durs, 95), 10)
bins = np.linspace(0, mx, 30)
axes[1].hist(soc_d[soc_d<mx], bins=bins, alpha=.7, color='#e74c3c', label='Social')
axes[1].hist(sol_d[sol_d<mx], bins=bins, alpha=.7, color='#3498db', label='Solo')
axes[1].set_xlabel('Duration (s)'); axes[1].set_ylabel('Count')
axes[1].set_title('Epoch duration distribution'); axes[1].legend()

show = min(len(ex_beh), int(120*IMAGING_FPS))
axes[2].fill_between(t[:show], ex_beh[:show].astype(float),
                     step='pre', alpha=.7, color='#e74c3c')
axes[2].set_xlabel('Time (s)'); axes[2].set_yticks([0,1])
axes[2].set_yticklabels(['Solo','Social'])
axes[2].set_title('Timeline (first 120s)')

plt.tight_layout(); plt.show()

---
## 6. Preliminary Spectral Analysis

At 30 fps, Nyquist = 15 Hz. Frequency bands:
- **Infraslow** 0.01-0.1 Hz | **Slow** 0.1-1 Hz | **Delta** 1-4 Hz | **Theta** 4-7 Hz

In [ ]:
FREQ_BANDS = {'infraslow': (0.01, 0.1), 'slow': (0.1, 1.0),
              'delta': (1.0, 4.0), 'theta': (4.0, 7.0)}
BAND_COLORS = {'infraslow': '#f39c12', 'slow': '#2ecc71',
               'delta': '#3498db', 'theta': '#9b59b6'}

print(f'fs={IMAGING_FPS} Hz, Nyquist={IMAGING_FPS/2} Hz')
for n,(lo,hi) in FREQ_BANDS.items():
    print(f'  {n:12s}: {lo}-{hi} Hz')

### 6.1 Welch PSD

In [ ]:
nperseg = int(10 * IMAGING_FPS)  # 10s windows
noverlap = nperseg // 2
nfft = max(1024, nperseg * 2)

print(f'Window: {nperseg/IMAGING_FPS:.0f}s, Overlap: 50%, FFT: {nfft}, '
      f'Resolution: {IMAGING_FPS/nperseg:.3f} Hz')

f_w, psd_all = welch(ex_cal.T, fs=IMAGING_FPS,
                     nperseg=nperseg, noverlap=noverlap, nfft=nfft, axis=1)
print(f'PSD: {psd_all.shape} (neurons x freq)')

In [ ]:
mpsd = psd_all.mean(axis=0)
spsd = psd_all.std(axis=0) / np.sqrt(psd_all.shape[0])

fig, axes = plt.subplots(1, 2, figsize=(16, 5))
for ax in axes:
    for n,(lo,hi) in FREQ_BANDS.items():
        ax.axvspan(lo, hi, alpha=0.1, color=BAND_COLORS[n], label=n)

axes[0].plot(f_w, mpsd, color='#2c3e50', lw=1.5)
axes[0].fill_between(f_w, mpsd-spsd, mpsd+spsd, alpha=.3, color='#3498db')
axes[0].set_xlabel('Freq (Hz)'); axes[0].set_ylabel('PSD')
axes[0].set_title('Mean PSD (linear)'); axes[0].legend(fontsize=8)

axes[1].loglog(f_w[1:], mpsd[1:], color='#2c3e50', lw=1.5)
axes[1].fill_between(f_w[1:], (mpsd-spsd)[1:], (mpsd+spsd)[1:], alpha=.3, color='#3498db')
axes[1].set_xlabel('Freq (Hz)'); axes[1].set_ylabel('PSD')
axes[1].set_title('Mean PSD (log-log)')

plt.tight_layout(); plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6))
fm = f_w <= 10
im = ax.imshow(10*np.log10(psd_all[:,fm]+1e-12), aspect='auto', cmap='viridis',
               extent=[f_w[fm][0], f_w[fm][-1], psd_all.shape[0], 0])
ax.set_xlabel('Freq (Hz)'); ax.set_ylabel('Neuron')
ax.set_title('PSD heatmap (dB)')
plt.colorbar(im, label='Power (dB)')
plt.tight_layout(); plt.show()

### 6.2 Spectrogram (STFT)

In [ ]:
active_n = np.argmax(ex_cal.std(axis=0))
sig = ex_cal[:, active_n]

sn = int(5 * IMAGING_FPS)
f_s, t_s, Zxx = stft(sig, fs=IMAGING_FPS, nperseg=sn, noverlap=sn//2)

fig, axes = plt.subplots(3, 1, figsize=(18, 10),
                         gridspec_kw={'height_ratios': [1, 3, 0.5]})

axes[0].plot(t, sig, lw=0.4, color='#2c3e50')
shade_social(axes[0], ex_beh, t)
axes[0].set_ylabel('Calcium')
axes[0].set_title(f'Neuron {active_n} - STFT Spectrogram')

fl = 10; fmask = f_s <= fl
axes[1].pcolormesh(t_s, f_s[fmask], 10*np.log10(np.abs(Zxx[fmask])**2+1e-12),
                   shading='gouraud', cmap='magma')
axes[1].set_ylabel('Freq (Hz)')
for _,(lo,hi) in FREQ_BANDS.items():
    if hi<=fl: axes[1].axhline(lo, color='white', lw=0.5, alpha=0.4, ls='--')

axes[2].fill_between(t, ex_beh.astype(float), step='pre', alpha=.7, color='#e74c3c')
axes[2].set_yticks([0,1]); axes[2].set_yticklabels(['Solo','Social'])
axes[2].set_xlabel('Time (s)')

plt.tight_layout(); plt.show()

### 6.3 Morlet Wavelet Scalogram (60s segment)

In [ ]:
seg = int(60 * IMAGING_FPS)
ss, ts, bs = sig[:seg], t[:seg], ex_beh[:seg]

freqs_want = np.linspace(0.5, 9, 50)
wn = 'cmor1.5-1.0'
scales = pywt.central_frequency(wn) * IMAGING_FPS / freqs_want

print(f'CWT: 60s, {len(freqs_want)} freqs...')
coeffs, freqs = pywt.cwt(ss, scales, wn, sampling_period=1/IMAGING_FPS)
power = np.abs(coeffs)**2

fig, axes = plt.subplots(2, 1, figsize=(18, 8), gridspec_kw={'height_ratios': [1, 3]})

axes[0].plot(ts, ss, lw=0.5, color='#2c3e50')
shade_social(axes[0], bs, ts)
axes[0].set_ylabel('Calcium')
axes[0].set_title(f'Neuron {active_n} - Morlet CWT (60s)')

axes[1].pcolormesh(ts, freqs, 10*np.log10(power+1e-12), shading='gouraud', cmap='magma')
axes[1].set_ylabel('Freq (Hz)'); axes[1].set_xlabel('Time (s)')
for _,(lo,hi) in FREQ_BANDS.items():
    axes[1].axhline(lo, color='white', lw=0.5, alpha=0.4, ls='--')

plt.tight_layout(); plt.show()
print('Done.')

---
## 7. Social vs. Solo: Spectral Feature Comparison

Extract features from 5-second windows within pure behavioral states across **all sessions**.

In [ ]:
def spectral_features(segment, fs, bands):
    """Spectral features from a 1D signal."""
    nl = min(len(segment), int(5*fs))
    f, psd = welch(segment, fs=fs, nperseg=nl, noverlap=nl//2)
    ft = {}

    for name,(lo,hi) in bands.items():
        m = (f>=lo)&(f<hi)
        ft[f'power_{name}'] = np.trapz(psd[m], f[m]) if m.sum()>0 else 0.0

    tp = np.trapz(psd[f>0], f[f>0])
    ft['total_power'] = tp
    if tp > 0:
        for name in bands:
            ft[f'relpower_{name}'] = ft[f'power_{name}'] / tp

    pn = psd[f>0] / (psd[f>0].sum()+1e-12)
    pn = pn[pn>0]
    ft['spectral_entropy'] = -np.sum(pn * np.log2(pn))
    ft['peak_freq'] = f[np.argmax(psd)]

    fp, pp = f[f>0], psd[f>0]
    ft['spectral_centroid'] = np.sum(fp*pp) / (np.sum(pp)+1e-12)

    cum = np.cumsum(pp)
    if cum[-1]>0:
        ei = np.searchsorted(cum, 0.9*cum[-1])
        ft['spectral_edge_90'] = fp[min(ei, len(fp)-1)]

    if ft.get('power_delta',0)>0:
        ft['theta_delta_ratio'] = ft.get('power_theta',0) / ft['power_delta']

    return ft

In [ ]:
W_SEC = 5.0
W_SAMP = int(W_SEC * IMAGING_FPS)
PURITY = 0.9

all_feat = []
for si in tqdm(range(len(aligned_calcium)), desc='Sessions'):
    cal = aligned_calcium[si]
    beh = aligned_behavior[si]

    for start in range(0, len(cal)-W_SAMP, W_SAMP):
        end = start + W_SAMP
        frac = beh[start:end].mean()
        if frac > PURITY:       label = 1
        elif frac < 1-PURITY:   label = 0
        else:                   continue

        pm = cal[start:end].mean(axis=1)
        ft = spectral_features(pm, IMAGING_FPS, FREQ_BANDS)
        ft['label'] = label
        ft['session'] = si
        ft['start_time'] = start / IMAGING_FPS
        ft['isolation'] = session_info[si]['isolation']
        all_feat.append(ft)

df_feat = pd.DataFrame(all_feat)
print(f'\nWindows: {len(df_feat)} '
      f'(social={(df_feat["label"]==1).sum()}, solo={(df_feat["label"]==0).sum()})')

In [ ]:
pf = [f for f in ['power_delta','power_theta','spectral_entropy',
      'spectral_centroid','theta_delta_ratio','total_power'] if f in df_feat.columns]

fig, axes = plt.subplots(2, (len(pf)+1)//2, figsize=(18, 9))
axes = axes.ravel()

for i, feat in enumerate(pf):
    soc = df_feat.loc[df_feat['label']==1, feat].dropna()
    sol = df_feat.loc[df_feat['label']==0, feat].dropna()
    axes[i].hist(sol, bins=30, alpha=.6, color='#3498db', label='Solo', density=True)
    axes[i].hist(soc, bins=30, alpha=.6, color='#e74c3c', label='Social', density=True)
    axes[i].set_title(feat, fontsize=11); axes[i].legend(fontsize=8)
    _, pv = mannwhitneyu(soc, sol, alternative='two-sided')
    sig = '***' if pv<.001 else '**' if pv<.01 else '*' if pv<.05 else 'ns'
    axes[i].set_xlabel(f'p={pv:.2e} {sig}', fontsize=9)

for j in range(len(pf), len(axes)): axes[j].set_visible(False)
plt.suptitle('Spectral Features: Social vs Solo (all sessions)', fontsize=14, y=1.01)
plt.tight_layout(); plt.show()

In [ ]:
# Mean PSD: social vs solo
soc_p, sol_p = [], []
for si in range(len(aligned_calcium)):
    cal=aligned_calcium[si]; beh=aligned_behavior[si]
    for start in range(0, len(cal)-W_SAMP, W_SAMP):
        end=start+W_SAMP; frac=beh[start:end].mean()
        pm=cal[start:end].mean(axis=1)
        nl=min(W_SAMP, int(5*IMAGING_FPS))
        fs_, ps_ = welch(pm, fs=IMAGING_FPS, nperseg=nl, noverlap=nl//2)
        if frac>PURITY: soc_p.append(ps_)
        elif frac<1-PURITY: sol_p.append(ps_)

soc_p=np.array(soc_p); sol_p=np.array(sol_p)
sm,ss=soc_p.mean(0), soc_p.std(0)/np.sqrt(len(soc_p))
om,os=sol_p.mean(0), sol_p.std(0)/np.sqrt(len(sol_p))

fig, axes = plt.subplots(1, 2, figsize=(16, 5))

axes[0].semilogy(fs_, sm, color='#e74c3c', lw=1.5, label=f'Social (n={len(soc_p)})')
axes[0].fill_between(fs_, sm-ss, sm+ss, alpha=.3, color='#e74c3c')
axes[0].semilogy(fs_, om, color='#3498db', lw=1.5, label=f'Solo (n={len(sol_p)})')
axes[0].fill_between(fs_, om-os, om+os, alpha=.3, color='#3498db')
for n,(lo,hi) in FREQ_BANDS.items(): axes[0].axvspan(lo,hi,alpha=.08,color=BAND_COLORS[n])
axes[0].set_xlabel('Freq (Hz)'); axes[0].set_ylabel('PSD')
axes[0].set_title('Mean PSD: Social vs Solo'); axes[0].legend()

lr = np.log2(sm / (om+1e-12))
axes[1].plot(fs_, lr, color='#2c3e50', lw=1.5)
axes[1].axhline(0, color='gray', ls='--', lw=.8)
for n,(lo,hi) in FREQ_BANDS.items(): axes[1].axvspan(lo,hi,alpha=.08,color=BAND_COLORS[n])
axes[1].set_xlabel('Freq (Hz)'); axes[1].set_ylabel('log2(Social/Solo)')
axes[1].set_title('PSD ratio (>0 = more power during social)')

plt.tight_layout(); plt.show()

---
## 8. Classification Sanity Check

**GroupKFold** with sessions as groups prevents cross-session data leakage.

In [ ]:
meta = ['label','session','start_time','isolation']
fcols = [c for c in df_feat.columns if c not in meta]

X = df_feat[fcols].fillna(0).values
y = df_feat['label'].values
groups = df_feat['session'].values

Xs = np.nan_to_num(StandardScaler().fit_transform(X))
ng = len(np.unique(groups))
ns = min(5, ng)
gkf = GroupKFold(n_splits=ns)

clfs = {'LDA': LinearDiscriminantAnalysis(),
        'SVM (linear)': SVC(kernel='linear', probability=True),
        'Logistic Reg': LogisticRegression(max_iter=1000)}

print(f'{ns}-fold GroupKFold (groups=sessions)')
print(f'  Samples: {len(y)} (social={(y==1).sum()}, solo={(y==0).sum()})')
print(f'  Features: {Xs.shape[1]}, Groups: {ng}')
print('='*60)

results = {}
for name, clf in clfs.items():
    accs, aucs = [], []
    for tri, tei in gkf.split(Xs, y, groups):
        ytr, yte = y[tri], y[tei]
        if len(np.unique(ytr))<2 or len(np.unique(yte))<2: continue
        clf.fit(Xs[tri], ytr)
        accs.append(accuracy_score(yte, clf.predict(Xs[tei])))
        aucs.append(roc_auc_score(yte, clf.predict_proba(Xs[tei])[:,1]))
    results[name] = {'acc':np.mean(accs), 'acc_std':np.std(accs),
                     'auc':np.mean(aucs), 'auc_std':np.std(aucs)}
    print(f'\n  {name}: Acc={np.mean(accs):.3f}+/-{np.std(accs):.3f}, '
          f'AUC={np.mean(aucs):.3f}+/-{np.std(aucs):.3f}')

In [ ]:
# Permutation test
N_PERM = 200
cp = LogisticRegression(max_iter=1000)
pa = []

for _ in tqdm(range(N_PERM), desc='Permutations'):
    ys = np.random.permutation(y)
    fa = []
    for tri,tei in gkf.split(Xs, ys, groups):
        ytr,yte = ys[tri], ys[tei]
        if len(np.unique(ytr))<2 or len(np.unique(yte))<2: continue
        cp.fit(Xs[tri], ytr)
        fa.append(accuracy_score(yte, cp.predict(Xs[tei])))
    if fa: pa.append(np.mean(fa))

pa = np.array(pa)
real = results['Logistic Reg']['acc']
pval = np.mean(pa >= real)

fig, ax = plt.subplots(figsize=(10, 4))
ax.hist(pa, bins=30, color='#95a5a6', alpha=.7, label='Null')
ax.axvline(real, color='#e74c3c', lw=2, ls='--', label=f'Real={real:.3f}')
ax.set_xlabel('Accuracy'); ax.set_ylabel('Count')
ax.set_title(f'Permutation test (p={pval:.4f})'); ax.legend()
plt.tight_layout(); plt.show()

print(f'Real: {real:.3f}, Chance: {pa.mean():.3f}+/-{pa.std():.3f}, p={pval:.4f}')
print('SIGNIFICANT' if pval<0.05 else 'Not significant at p<0.05')

---
## 9. Summary & Next Steps

### Key findings (fill in after running)

| Question | Finding |
|----------|---------|
| Sessions loaded | X / 18 aligned |
| Neurons per session | range |
| Class balance | X% social vs Y% solo |
| Photobleaching | present / absent |
| PSD structure | 1/f? band peaks? |
| Social vs Solo PSD | differences in which bands? |
| Classification | Acc X% (chance Y%, p=Z) |

### Next steps

1. **Preprocessing** — per-neuron detrending, z-scoring
2. **Per-neuron features** — which neurons are most discriminative?
3. **Isolation condition** — spectral signatures by GH / 24hr / 7d?
4. **Wavelet features** — time-resolved spectral features from CWT
5. **Movement confound** — SLEAP pose data could control for locomotion
6. **Final pipeline** — optimized features, proper CV, publication figures